## Importações e inicialização do Spark

In [ ]:
import os

# coloca as variáveis de ambiente necessárias para o PySpark no windows
os.environ['JAVA_HOME'] = r"C:\Program Files\Eclipse Adoptium\jdk-8.0.452.9-hotspot"
os.environ['SPARK_HOME'] = r"C:\spark\spark-3.5.6-bin-hadoop3"

os.environ['PYSPARK_PYTHON'] = r"C:\Users\diego\Desktop\spark_alura\.venv\Scripts\python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = r"C:\Users\diego\Desktop\spark_alura\.venv\Scripts\python.exe"


In [ ]:
import findspark # busca o PySpark no sistema
findspark.init()

In [ ]:
# importa a classe que inicia uma sessão do Spark para usar o Spark SQL
from pyspark.sql import SparkSession

In [ ]:
# inicia uma sessão do Spark, que é o ponto de entrada para usar o Spark SQL
spark = SparkSession.builder.master('local[*]').appName("Iniciando").config("spark.hadoop.io.native.lib", "false").getOrCreate()
spark

## Exemplos de DataFrames

In [ ]:
# exemplo de criação de um DataFrame usando tuplas
data = [('zeca', '35'), ('eva', '29')]
col_names = ['nome', 'idade']

df = spark.createDataFrame(data, col_names)

df.show()

In [ ]:
# exemplo de criação de um DataFrame usando dicionários
data_dict = [{'nome': 'zeca', 'idade': '35'}, {'nome': 'eva', 'idade': '29'}]

df = spark.createDataFrame(data_dict)

df.show()

In [ ]:
# converte o DataFrame do Spark para um DataFrame do Pandas
df.toPandas()

## Extrair conjunto de dados

In [ ]:
zip_path = os.path.abspath(r'.\zip_files')
data_path = os.path.abspath(r'.\data')

In [ ]:
import zipfile # biblioteca nativa do Python para manipular arquivos zip

zipfile.ZipFile(os.path.join(zip_path, 'empresas.zip'), 'r').extractall(data_path)
zipfile.ZipFile(os.path.join(zip_path, 'estabelecimentos.zip'), 'r').extractall(data_path)
zipfile.ZipFile(os.path.join(zip_path, 'socios.zip'), 'r').extractall(data_path)

In [ ]:
# retorna uma lista com os caminhos dos arquivos de um diretório
def get_files_from_directory(directory):
    files = [
        os.path.join(directory, f)
        for f in os.listdir(directory)
    ]
    return files

In [ ]:
# lê todos os arquivos CSV do diretório especificado e cria um DataFrame
df_emps = spark.read.csv(
    get_files_from_directory(os.path.join(data_path, 'empresas')), 
    sep=";", header=False, inferSchema=True)

df_emps.count()

In [ ]:
df_estabs = spark.read.csv(
    get_files_from_directory(os.path.join(data_path, 'estabelecimentos')), 
    sep=";", header=False, inferSchema=True)

df_estabs.count()

In [ ]:
df_socios = spark.read.csv(
    get_files_from_directory(os.path.join(data_path, 'socios')), 
    sep=";", header=False, inferSchema=True)

df_socios.count()

In [ ]:
# exibe as primeiras 3 linhas dos DataFrames
df_emps.show(3)
df_estabs.show(3)
df_socios.show(3)

In [ ]:
# converte as primeiras 3 linhas do DataFrame do Spark para um DataFrame do Pandas
df_socios.limit(3).toPandas()

## Renomear colunas

In [ ]:
# define os nomes das colunas para os DataFrames

emps_col_names = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']

estabs_col_names = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']

socios_col_names = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

In [ ]:
# exibe os nomes das colunas dos DataFrames em formato enumerado para mapeamento das colunas quando renomear
for i, col_name in enumerate(emps_col_names):
    print([i, col_name])

In [ ]:
# função para renomear as colunas de um DataFrame utilizando o mapeamento de nomes de colunas
def rename_columns(df, col_names):
    for i, col_name in enumerate(col_names):
        df = df.withColumnRenamed(f'_c{i}', col_name)
    return df

In [ ]:
# renomeia as colunas dos DataFrames
df_emps = rename_columns(df_emps, emps_col_names)
df_estabs = rename_columns(df_estabs, estabs_col_names)
df_socios = rename_columns(df_socios, socios_col_names)

In [ ]:
# exibe as primeiras 3 linhas dos DataFrames com os nomes das colunas renomeados
print(df_emps.columns)
print(df_estabs.columns)
print(df_socios.columns)

In [ ]:
# converte as primeiras 3 linhas do DataFrame do Spark para DataFrame do Pandas com o nomes das colunas renomeados
df_socios.limit(3).toPandas()

# Converter para coluna de dinheiro para tipo Double

In [ ]:
# mostra os nomes das colunas e seus tipos de dados
df_emps.printSchema()
df_socios.printSchema()
df_estabs.printSchema()

In [ ]:
# importa a classe DoubleType para converter o tipo de dado para Double
# importa a classe functions do para manipulação de colunas
from pyspark.sql.types import DoubleType
from pyspark.sql import functions as fx


In [ ]:
df_emps.show(3)

In [ ]:
# substitui a vírgula por ponto na coluna 'capital_social_da_empresa' para que o PySpark reconheça como número decimal
df_emps = df_emps.withColumn('capital_social_da_empresa', fx.regexp_replace('capital_social_da_empresa', ',', '.'))
df_emps.select('capital_social_da_empresa').show(3)

In [ ]:
df_emps.printSchema()

In [ ]:
# converte a coluna 'capital_social_da_empresa' para o tipo Double
df_emps = df_emps.withColumn('capital_social_da_empresa', df_emps['capital_social_da_empresa'].cast(DoubleType()))
df_emps.printSchema()

# Converter coluna para o tipo Date

In [ ]:
# importa a classe StringType para converter o tipo de dado para String
from pyspark.sql.types import StringType

In [ ]:
# a coluna estava com tipo de dado Integer, mas foi convertida para String para que seja possivel passar to_date
df_socios = df_socios.withColumn('data_de_entrada_sociedade', df_socios['data_de_entrada_sociedade'].cast(StringType()))
df_socios.printSchema()

In [ ]:
df_socios.select('data_de_entrada_sociedade').show(3)

In [ ]:
# para converter para o tipo Date é necessario converter para String primeiro, depois converte para Date usando o mesmo formato de data na coluna original
df_socios = df_socios.withColumn('data_de_entrada_sociedade', fx.to_date(df_socios['data_de_entrada_sociedade'], 'yyyyMMdd'))

df_socios.printSchema()

In [ ]:
df_socios.show(3)

# Converter multiplas colunas em tipo Date

In [ ]:
df_estabs.select('data_situacao_cadastral', 'data_de_inicio_atividade', 'data_da_situacao_especial').show(3)

In [ ]:
# aplica a conversão de data para várias colunas do DataFrame de estabelecimentos
df_estabs = df_estabs\
    .withColumn('data_situacao_cadastral', fx.to_date(df_estabs.data_situacao_cadastral\
        .cast(StringType()), 'yyyyMMdd'))\
    .withColumn('data_de_inicio_atividade', fx.to_date(df_estabs.data_de_inicio_atividade\
        .cast(StringType()), 'yyyyMMdd'))\
    .withColumn('data_da_situacao_especial', fx.to_date(df_estabs.data_da_situacao_especial\
        .cast(StringType()), 'yyyyMMdd'))

df_estabs.select('data_situacao_cadastral', 'data_de_inicio_atividade', 'data_da_situacao_especial').show(3)

In [ ]:
df_estabs.select('data_situacao_cadastral', 'data_de_inicio_atividade', 'data_da_situacao_especial').printSchema()

## Extrair ano de uma coluna de data

In [ ]:
# assim como no SQL, podemos aplicar funções de transformação nas colunas
df_socios\
    .select(
        'nome_do_socio_ou_razao_social',
        fx.year('data_de_entrada_sociedade').alias('ano_entrada_sociedade'), # extrai o ano da data e renomeia a coluna
    )\
    .show(5, truncate=False) # 'truncate=False' para mostrar o texto completo das colunas

### Exercício - Extrair parte de string de uma coluna:
- Criar coluna com ultimo sobrenome e primeiro nome, separados por virgula

In [ ]:
data = [
    ('GISELLE PAULA GUIMARAES CASTRO', 15),
    ('ELAINE GARCIA DE OLIVEIRA', 22),
    ('JOAO CARLOS ABNER DE LOURDES', 43),
    ('MARTA ZELI FERREIRA', 24),
    ('LAUDENETE WIGGERS ROEDER', 51)
]
col_names = ['nome', 'idade']

df = spark.createDataFrame(data, col_names)
df.show(truncate=False)

In [ ]:
# cria uma coluna com o último sobrenome e o primeiro nome, separados por vírgula
df\
    .select(
        'nome',
        fx.concat_ws( # concatena as colunas com um separador
            ', ', 
            fx.substring_index('nome', ' ', -1), # pega o ultimo sobrenome
            fx.substring_index('nome', ' ', 1) # pega o primeiro nome
        ).alias('ident'), 
        'idade'
    )\
    .show(truncate=False)